# Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import welch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/wearable_eeg/Data/"

# Load Data

In [ ]:
# Load RT, X and y back
RT = pd.read_csv(base_path+"RT.csv").values
X = pd.read_csv(base_path+"X.csv").values
y = pd.read_csv(base_path+"y.csv")["y"].values

print("RT shape:", RT.shape)
print("X shape:", X.shape)
print("y shape:", y.shape)

RT shape: (900, 1)
X shape: (900, 5362)
y shape: (900,)


In [ ]:
n_channels = 14
fs = int(128)
pre = int(0.1 * 128)   # 0.1s baseline
post = int(2.9 * 128)  # 2.9s after
window_len = pre + post
n_times = window_len

In [ ]:
# Reshape X into (n_samples, n_times, n_channels)
X = X.reshape(-1, n_times, n_channels)

In [ ]:
X.shape # (900, 383, 14)

(900, 383, 14)

# Get P300 and N400

In [ ]:
def peak_and_latency(trial, win, fs=128):
    if isinstance(win, slice):
        start, end = win.start, win.stop
    else:
        start, end = win

    seg = trial[start:end, :]  # (time, channels)
    if seg.shape[0] == 0:  # empty window check
        return np.zeros(trial.shape[1]), np.zeros(trial.shape[1])

    peak_amp = seg.max(axis=0)  # (channels,)
    latency = seg.argmax(axis=0) / fs + start/fs
    return peak_amp, latency

    return peak_amp, latency

def auc_feature(trial, win):
    if isinstance(win, slice):
        start, end = win.start, win.stop
    else:
        start, end = win

    seg = trial[start:end, :]  # (time, channels)
    if seg.shape[0] == 0:
        return np.zeros(trial.shape[1])

    return np.trapezoid(seg, axis=0)  # (channels,)

def erp_diff(trial, win, pre_win):
    if isinstance(win, slice):
        start, end = win.start, win.stop
    else:
        start, end = win

    if isinstance(pre_win, slice):
        pre_start, pre_end = pre_win.start, pre_win.stop
    else:
        pre_start, pre_end = pre_win

    # mean across time (axis=0), keep per channel
    post_mean = trial[start:end, :].mean(axis=0)  # (channels,)
    pre_mean  = trial[pre_start:pre_end, :].mean(axis=0)  # (channels,)

    return post_mean - pre_mean  # (channels,)

In [ ]:
p300_win = slice(pre+32, pre+45)   # stimulus onset + offset
n400_win = slice(pre+45, pre+58)
pre_win  = slice(0, pre)

In [ ]:
all_feats = []
for trial in X:  # trial: (time, channels)
    trial_feats = []

    # baseline mean per channel
    baseline = trial[0:pre, :].mean(axis=0)  # (channels,)
    trial_feats.extend(baseline.tolist())

    # P300
    peak, lat = peak_and_latency(trial, p300_win)
    trial_feats.extend(peak.tolist() + lat.tolist())
    trial_feats.extend(auc_feature(trial, p300_win).tolist())
    trial_feats.extend(erp_diff(trial, p300_win, pre_win).tolist())

    # N400
    peak, lat = peak_and_latency(trial, n400_win)
    trial_feats.extend(peak.tolist() + lat.tolist())
    trial_feats.extend(auc_feature(trial, n400_win).tolist())
    trial_feats.extend(erp_diff(trial, n400_win, pre_win).tolist())

    all_feats.append(trial_feats)

erp_feats = np.array(all_feats)
print("ERP feature matrix:", erp_feats.shape)

ERP feature matrix: (900, 126)


In [ ]:
# min as feature
p300_min = X[:, p300_win, :].min(axis=1)  # (trials, channels)
n400_min = X[:, n400_win, :].min(axis=1)  # (trials, channels)

# mean as feature
p300_mean = X[:, p300_win, :].mean(axis=1)  # (trials, channels)
n400_mean = X[:, n400_win, :].mean(axis=1)  # (trials, channels)

# max as feature
p300_max = X[:, p300_win, :].max(axis=1)  # (trials, channels)
n400_max = X[:, n400_win, :].max(axis=1)  # (trials, channels)


p300_feats = np.concat([p300_min,p300_mean,p300_max], axis=1)
n400_feats = np.concat([n400_min,n400_mean,n400_max], axis=1)

In [ ]:
print(p300_feats.shape)
print(n400_feats.shape)

(900, 42)
(900, 42)


# Get PSD on the frequency bands

In [ ]:
bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta":  (13, 30),
    "gamma": (30, 45)
}

def relative_bandpower(signal, fs=128):
    f, Pxx = welch(signal, fs=fs, nperseg=min(256, len(signal)))
    total_power = np.trapezoid(Pxx, dx=f[1]-f[0])
    feats = []
    for band in bands.values():
        idx = np.logical_and(f >= band[0], f <= band[1])
        band_power = np.trapezoid(Pxx[idx], dx=f[1]-f[0])
        feats.append(band_power / total_power if total_power > 0 else 0)
    return feats

psd_feats = []
for trial in X:
    trial_feats = []
    for ch in range(trial.shape[1]):
        pre_feats  = relative_bandpower(trial[pre_win, ch])
        post_feats = relative_bandpower(trial[pre:pre+post, ch])  # 2.9s after stim
        trial_feats.extend(pre_feats + post_feats)
    psd_feats.append(trial_feats)

psd_feats = np.array(psd_feats)
psd_feats.shape


(900, 140)

# X_features

In [ ]:
# Final matrix = ERP + PSD
X_features = np.hstack([p300_feats, n400_feats, erp_feats, psd_feats])
print("Final feature matrix:", X_features.shape)

Final feature matrix: (900, 350)


In [ ]:
pd.DataFrame(X_features).to_csv(base_path+"X_features.csv", index=False)

# X_final

In [ ]:
from scipy.signal import welch
from scipy.stats import entropy
import librosa

# === PSD on first second ===
def compute_psd_first_second(trial, fs=128, bands=None):
    if bands is None:
        bands = {
            "delta": (0.1, 4),
            "theta": (4, 8),
            "alpha": (8, 13),
            "beta": (13, 32),
            "gamma": (32, 50)
        }
    first_sec = trial[:fs, :]  # only first second
    feats = []
    for ch in range(first_sec.shape[1]):
        f, Pxx = welch(first_sec[:, ch], fs=fs, nperseg=min(fs*2, first_sec.shape[0]))
        total_power = np.trapezoid(Pxx, f) + 1e-12
        for (low, high) in bands.values():
            idx = (f >= low) & (f <= high)
            band_power = np.trapezoid(Pxx[idx], f[idx])
            feats.append(band_power / total_power)
    return np.array(feats)


# === Spectral entropy on quarters of first second ===
def compute_spectral_entropy_first_second(trial, fs=128, n_splits=4):
    first_sec = trial[:fs, :]
    seg_len = fs // n_splits
    feats = []
    for ch in range(first_sec.shape[1]):
        for i in range(n_splits):
            seg = first_sec[i*seg_len:(i+1)*seg_len, ch]
            f, Pxx = welch(seg, fs=fs, nperseg=len(seg))
            Pxx_norm = Pxx / (np.sum(Pxx) + 1e-12)
            feats.append(entropy(Pxx_norm))
    return np.array(feats)


# === LPC ===
def compute_lpc(trial, order=10):
    feats = []
    for ch in range(trial.shape[1]):
        try:
            lpc = librosa.lpc(trial[:, ch], order=order)
        except Exception:
            lpc = np.zeros(order+1)
        feats.extend(lpc[1:])  # skip a0
    return np.array(feats)

# === ERP AUC + Energy ===
def compute_auc_energy(seg, n_channels=None):
    """
    Return [Energy, AUC] per channel for a given segment.
    If n_channels is provided, pad to that length.
    """
    if seg.shape[0] == 0:  # safeguard for empty slice
        return np.zeros((n_channels or seg.shape[1]) * 2)

    energy = (seg**2).sum(axis=0)
    auc = np.trapezoid(seg, axis=0)
    feats = np.concatenate([energy, auc])

    # pad if needed (e.g., if seg had fewer channels after selection)
    if n_channels is not None and len(feats) < n_channels * 2:
        feats = np.pad(feats, (0, n_channels * 2 - len(feats)), constant_values=0)

    return feats

# === Response Time ===
def compute_response_time(stim_onset, resp_onset, fs=128):
    return np.array([(resp_onset - stim_onset) / fs])

# === Final extractor ===
def extract_features_M1_AUC_Energy(trial, fs=128, stim_onset=0.1, resp_onset=None):
    feats = []

    # === Matrix 1 ===
    feats.extend(compute_psd_first_second(trial, fs))                # PSD
    feats.extend(compute_spectral_entropy_first_second(trial, fs))   # SE
    feats.extend(compute_lpc(trial))                    # LPC
    if resp_onset is not None:
        # feats.append(compute_response_time(stim_onset, resp_onset, fs)) # in samples
        feats.extend(resp_onset) # already in seconds

    # === ERP energy + AUC (all channels) ===
    feats.extend(compute_auc_energy(trial, n_channels=trial.shape[1]))

    # P300 & N400 restricted
    p300_win = slice(int((stim_onset + 0.25) * fs), int((stim_onset + 0.35) * fs))
    n400_win = slice(int((stim_onset + 0.35) * fs), int((stim_onset + 0.45) * fs))

    target_ch = [3, 10]  # only ch4 and ch11

    # P300
    seg_p300 = trial[p300_win, target_ch]                # only ch4 and ch11
    feats.extend(compute_auc_energy(seg_p300))

    # N400
    seg_n400 = trial[n400_win, target_ch]                # only ch4 and ch11
    feats.extend(compute_auc_energy(seg_n400))

    # Final cleanup (replace NaN/Inf by 0)
    feats = np.nan_to_num(feats, nan=0.0, posinf=0.0, neginf=0.0)

    return np.array(feats)

In [ ]:
def get_feature_names(n_channels, fs=128, n_lpc=10, n_splits=4):
    names = []

    # PSD
    bands = ["delta", "theta", "alpha", "beta", "gamma"]
    for ch in range(n_channels):
        for band in bands:
            names.append(f"PSD_{band}_Ch{ch+1}")

    # Spectral Entropy
    for ch in range(n_channels):
        for q in range(n_splits):
            names.append(f"SE_Q{q+1}_Ch{ch+1}")

    # LPC
    for ch in range(n_channels):
        for k in range(n_lpc):
            names.append(f"LPC_C{k+1}_Ch{ch+1}")

    # Response Time
    names.append("RT")

    # ERP Energy + AUC (all channels)
    for ch in range(n_channels):
        names.append(f"ERP_Energy_Ch{ch+1}")
        names.append(f"ERP_AUC_Ch{ch+1}")

    # P300 & N400 restricted
    target_ch = [3, 10]  # only ch4 and ch11
    for ch in target_ch:
        names.append(f"P300_Energy_Ch{ch+1}")   # only ch4 and ch11
        names.append(f"P300_AUC_Ch{ch+1}")     # only ch4 and ch11
        names.append(f"N400_Energy_Ch{ch+1}")  # only ch4 and ch11
        names.append(f"N400_AUC_Ch{ch+1}")     # only ch4 and ch11

    return names

In [ ]:
all_feats=[]
for trial_idx, trial in enumerate(X):  # X shape (n_trials, time, channels)
    feat_vector = extract_features_M1_AUC_Energy(trial, fs=128, stim_onset=0.1, resp_onset=RT[trial_idx])
    all_feats.append(feat_vector)

X_final = np.array(all_feats)
print("Final feature matrix:", X_final.shape)  # expect ~300 features

Final feature matrix: (900, 303)


In [ ]:
feature_names = get_feature_names(n_channels=X.shape[2])
X_final = np.array(all_feats)
df_features = pd.DataFrame(X_final, columns=feature_names)

In [ ]:
df_features

,PSD_delta_Ch1,PSD_theta_Ch1,PSD_alpha_Ch1,PSD_beta_Ch1,PSD_gamma_Ch1,PSD_delta_Ch2,PSD_theta_Ch2,PSD_alpha_Ch2,PSD_beta_Ch2,PSD_gamma_Ch2,...,ERP_Energy_Ch14,ERP_AUC_Ch14,P300_Energy_Ch4,P300_AUC_Ch4,N400_Energy_Ch4,N400_AUC_Ch4,P300_Energy_Ch11,P300_AUC_Ch11,N400_Energy_Ch11,N400_AUC_Ch11
0,0.584436,0.163271,0.036161,0.100595,0.066733,0.619334,0.104071,0.048717,0.098789,0.054241,...,-30.433009,-34.904184,0.700491,0.075915,2.192546,-0.185435,0.788279,0.038866,2.673637,-0.013454
1,0.256031,0.269023,0.146727,0.170858,0.033061,0.449877,0.013748,0.011275,0.037521,0.010703,...,64.025221,9.739109,3.540884,1.075746,-6.268833,3.441219,3.679766,1.358077,-6.363026,3.896657
2,0.484580,0.185753,0.041461,0.122867,0.010597,0.339368,0.029923,0.108739,0.225147,0.025827,...,11.356716,4.971700,0.030781,0.018886,0.471681,0.306862,0.074482,0.023619,0.886769,0.488797
3,0.313761,0.183488,0.082385,0.200044,0.030412,0.161684,0.083821,0.130628,0.388175,0.166831,...,17.060054,7.476612,0.039903,0.028046,-0.649837,0.443999,0.052271,0.011611,-0.740622,0.224707
4,0.057024,0.195452,0.208092,0.412442,0.077169,0.221954,0.155731,0.272907,0.196309,0.088272,...,-5.219148,-20.386565,0.021542,0.036446,0.392545,-0.497252,0.047049,0.044633,0.693205,-0.594728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.068247,0.197897,0.104256,0.272419,0.281493,0.166003,0.109932,0.064768,0.381894,0.207044,...,8.786373,0.000000,0.065177,0.035417,0.782985,0.309056,0.062687,0.109888,0.787212,1.012633
896,0.211642,0.247173,0.060200,0.332218,0.147624,0.062447,0.228103,0.043030,0.375715,0.255305,...,13.746136,0.000000,0.022139,0.028131,-0.362530,0.226875,0.021387,0.103447,-0.314796,0.941195
897,0.294060,0.042523,0.055180,0.440553,0.107901,0.133693,0.083575,0.100279,0.546778,0.098238,...,-6.738038,0.000000,0.014656,0.020494,-0.298376,-0.346713,0.014528,0.023625,0.261177,-0.114280
898,0.275411,0.095278,0.047260,0.367211,0.155489,0.161011,0.076314,0.040167,0.441518,0.274025,...,-4.059612,0.000000,0.004489,0.012913,-0.159361,-0.156861,0.007763,0.018993,0.125258,0.408365


In [ ]:
pd.DataFrame(df_features).to_csv(base_path+"X_final.csv", index=False)